In [1]:
from param import ml_names, pers_names, data_name, data_path, results_path, thres_sel_feat
from ml_models import loocv_day_bench, var_dis
from ml_models import MAE, MSE, RMSE, MAPE, R2
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(data_path)
df_ml_input_new = pd.read_csv('ml_input_day.csv', index_col = 0)

## Predict variance/distance

In [3]:
df_results = var_dis(df_ml_input_new)
df_results.to_csv('person_dis.csv')

## Predict mean benchmark

In [3]:
ls_results = list()
for pers_name in pers_names:
    ls_results_loocv = loocv_day_bench(df_ml_input_new, pers_name)
    ls_results.extend(ls_results_loocv)

In [4]:
df_results = pd.DataFrame(ls_results, columns = ['Personality', 'Participant ID', 'Y test', 'Y pred'])
df_results.head()

,Personality,Participant ID,Y test,Y pred
0,neoConscientiousness,181,[3.166666667],[2.8022204897094594]
1,neoConscientiousness,182,[2.75],[2.8054977754581385]
2,neoConscientiousness,191,[3.25],[2.800870495682457]
3,neoConscientiousness,198,[2.541666667],[2.806817526238318]
4,neoConscientiousness,200,[3.125],[2.8022097930312175]


In [5]:
ls_evalutation = list()
df_results_groups = df_results.groupby('Personality')
df_results_group_names = df_results_groups.size().index
for df_results_group_name in df_results_group_names:
    ls_evaluation_row = [df_results_group_name]
    df_results_group = df_results_groups.get_group(df_results_group_name)
    ls_test_vals = list()
    ls_pred_vals = list()
    for val in df_results_group['Y test'].tolist():
        if len(val) != 1:
            print('Error')
        
        ls_test_vals.append(val[0])

    for val in df_results_group['Y pred'].tolist():
        if len(val) != 1:
            print('Error')
            
        ls_pred_vals.append(val[0])
    
    df_evaluation_input = pd.DataFrame()
    df_evaluation_input['Y_test'] = ls_test_vals
    df_evaluation_input['Y_pred'] = ls_pred_vals
    
#     df_evalution_output_xgb = evaluate(df_evaluation_input_xgb)
    ls_evalutation.append([df_results_group_name, 
                               MAE(ls_pred_vals, ls_test_vals),
                               MSE(ls_pred_vals, ls_test_vals)])

In [6]:
df_results_summary = pd.DataFrame(ls_evalutation, columns = ['Personality', 'MAE', 'MSE'])

In [11]:
df_results_summary.to_csv('day_benchmark.csv')